# Frohlich Experiment Analysis

This notebook analyzes the experimental logs from the hypothesis_2_&_4/logs directory and generates visualizations for:
- Sankey chart: Initial vs Final preferences
- Pie chart: Agreement frequency
- Bar chart: Agreement by principle
- Bar chart: Model type preference analysis

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from pathlib import Path
import re

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully")

Libraries imported successfully


## Data Extraction and Processing

In [2]:
def extract_data_from_logs():
    """
    Extract data from all JSON log files in the logs directory
    Each row represents one agent from one experimental run
    """
    logs_dir = Path('logs')
    data = []
    
    # Get all JSON files in logs directory
    json_files = list(logs_dir.glob('*.json'))
    print(f"Found {len(json_files)} log files")
    
    for file_path in json_files:
        try:
            with open(file_path, 'r') as f:
                log_data = json.load(f)
            
            # Extract general experiment info
            general_info = log_data.get('general_information', {})
            config_file = general_info.get('config_file_used', 'unknown')
            consensus_reached = general_info.get('consensus_reached', False)
            agreed_principle = general_info.get('consensus_principle', None)
            
            # Extract agent data
            agents = log_data.get('agents', [])
            
            for agent in agents:
                # Get agent basic info
                agent_name = agent.get('name', 'unknown')
                model = agent.get('model', 'unknown')
                temperature = agent.get('temperature', 0)
                personality = agent.get('personality', 'unknown')
                
                # Get initial preference (from phase 1 initial ranking)
                phase1 = agent.get('phase_1', {})
                initial_ranking = phase1.get('initial_ranking', {}).get('ranking_result', {})
                rankings = initial_ranking.get('rankings', [])
                
                # Find the top-ranked principle (rank 1)
                initial_preference = None
                for ranking in rankings:
                    if ranking.get('rank') == 1:
                        initial_preference = ranking.get('principle')
                        break
                
                # Get final preference from general_info vote results
                final_vote_results = general_info.get('final_vote_results', {})
                final_preference = final_vote_results.get(agent_name, None)
                
                # Count number of agents and rounds from config info
                num_agents = len(agents)
                
                # Extract rounds info (assuming from demonstrations)
                demonstrations = phase1.get('demonstrations', [])
                num_rounds = len(demonstrations)
                
                # Map principle names to numbers for consistency with example.csv
                principle_mapping = {
                    'maximizing_floor': 1,
                    'maximizing_average': 2, 
                    'maximizing_average_floor_constraint': 3,
                    'maximizing_average_range_constraint': 4
                }
                
                initial_pref_num = principle_mapping.get(initial_preference, None)
                final_pref_num = principle_mapping.get(final_preference, None)
                agreed_principle_num = principle_mapping.get(agreed_principle, None)
                
                # Create row for this agent
                row = {
                    'Config': config_file,
                    'Rounds': num_rounds,
                    '#Agents': num_agents,
                    'Prompt': personality,
                    'Model own': model,
                    'Model other agents': str([a.get('model', 'unknown') for a in agents if a.get('name') != agent_name]),
                    'Temperature self': temperature,
                    'Initial Preference': initial_pref_num,
                    'Initial Preference Name': initial_preference,
                    'Final Preference': final_pref_num,
                    'Final Preference Name': final_preference,
                    'Agreement?': 'Yes' if consensus_reached else 'No',
                    'Agreed Principle': agreed_principle_num,
                    'Agreed Principle Name': agreed_principle,
                    'Agent Name': agent_name,
                    'File': file_path.stem
                }
                
                data.append(row)
                
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue
    
    return pd.DataFrame(data)

# Extract data
df = extract_data_from_logs()
print(f"Extracted data for {len(df)} agent instances from {df['File'].nunique()} experiments")
print(f"Shape: {df.shape}")
df.head()

Found 10 log files
Extracted data for 50 agent instances from 10 experiments
Shape: (50, 16)


,Config,Rounds,#Agents,Prompt,Model own,Model other agents,Temperature self,Initial Preference,Initial Preference Name,Final Preference,Final Preference Name,Agreement?,Agreed Principle,Agreed Principle Name,Agent Name,File
0,default_config.yaml,4,5,You are an american college student.,anthropic/claude-sonnet-4,"['anthropic/claude-sonnet-4', 'moonshotai/kimi...",0.0,3,maximizing_average_floor_constraint,3.0,maximizing_average_floor_constraint,Yes,3.0,maximizing_average_floor_constraint,Agent_1,config_03_results_20250811_035202
1,default_config.yaml,4,5,You are an american college student.,anthropic/claude-sonnet-4,"['anthropic/claude-sonnet-4', 'moonshotai/kimi...",0.0,3,maximizing_average_floor_constraint,3.0,maximizing_average_floor_constraint,Yes,3.0,maximizing_average_floor_constraint,Agent_2,config_03_results_20250811_035202
2,default_config.yaml,4,5,You are an american college student.,moonshotai/kimi-k2,"['anthropic/claude-sonnet-4', 'anthropic/claud...",0.0,3,maximizing_average_floor_constraint,3.0,maximizing_average_floor_constraint,Yes,3.0,maximizing_average_floor_constraint,Agent_3,config_03_results_20250811_035202
3,default_config.yaml,4,5,You are an american college student.,anthropic/claude-sonnet-4,"['anthropic/claude-sonnet-4', 'anthropic/claud...",0.0,3,maximizing_average_floor_constraint,3.0,maximizing_average_floor_constraint,Yes,3.0,maximizing_average_floor_constraint,Agent_4,config_03_results_20250811_035202
4,default_config.yaml,4,5,You are an american college student.,meta-llama/llama-4-maverick,"['anthropic/claude-sonnet-4', 'anthropic/claud...",0.0,3,maximizing_average_floor_constraint,3.0,maximizing_average_floor_constraint,Yes,3.0,maximizing_average_floor_constraint,Agent_5,config_03_results_20250811_035202


In [3]:
# Display basic statistics
print("=== Data Summary ===")
print(f"Total agents: {len(df)}")
print(f"Total experiments: {df['File'].nunique()}")
print(f"Models used: {df['Model own'].unique()}")
print(f"Agreement rate: {(df['Agreement?'] == 'Yes').mean():.1%}")
print()

# Show data types and missing values
print("=== Data Info ===")
df.info()

=== Data Summary ===
Total agents: 50
Total experiments: 10
Models used: ['anthropic/claude-sonnet-4' 'moonshotai/kimi-k2'
 'meta-llama/llama-4-maverick' 'openai/gpt-4.1-mini'
 'deepseek/deepseek-chat-v3-0324' 'google/gemini-2.5-flash'
 'qwen/qwen3-coder' 'google/gemini-2.5-pro' 'x-ai/grok-4' 'openai/gpt-4.1']
Agreement rate: 80.0%

=== Data Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Config                   50 non-null     object 
 1   Rounds                   50 non-null     int64  
 2   #Agents                  50 non-null     int64  
 3   Prompt                   50 non-null     object 
 4   Model own                50 non-null     object 
 5   Model other agents       50 non-null     object 
 6   Temperature self         50 non-null     float64
 7   Initial Preference       50 non-null     int64  
 8  

## 1. Sankey Chart: Initial vs Final Preferences

In [4]:
def create_sankey_chart(df):
    """
    Create Sankey diagram showing flow from initial to final preferences
    """
    # Filter out rows with missing preferences
    df_clean = df.dropna(subset=['Initial Preference Name', 'Final Preference Name'])
    
    # Principle name mapping for display
    principle_names = {
        'maximizing_floor': 'Floor Income',
        'maximizing_average': 'Average Income', 
        'maximizing_average_floor_constraint': 'Average + Floor Constraint',
        'maximizing_average_range_constraint': 'Average + Range Constraint'
    }
    
    # Map principle names
    df_clean['Initial_Display'] = df_clean['Initial Preference Name'].map(principle_names)
    df_clean['Final_Display'] = df_clean['Final Preference Name'].map(principle_names)
    
    # Create flow data
    flows = df_clean.groupby(['Initial_Display', 'Final_Display']).size().reset_index(name='count')
    
    # Create node lists
    initial_nodes = [f"Initial: {name}" for name in principle_names.values()]
    final_nodes = [f"Final: {name}" for name in principle_names.values()]
    all_nodes = initial_nodes + final_nodes
    
    # Create node index mapping
    node_indices = {node: i for i, node in enumerate(all_nodes)}
    
    # Prepare source, target, and value lists
    source = []
    target = []
    value = []
    
    for _, row in flows.iterrows():
        initial_key = f"Initial: {row['Initial_Display']}"
        final_key = f"Final: {row['Final_Display']}"
        
        if initial_key in node_indices and final_key in node_indices:
            source.append(node_indices[initial_key])
            target.append(node_indices[final_key])
            value.append(row['count'])
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_nodes,
            color=["lightblue" if "Initial" in node else "lightcoral" for node in all_nodes]
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            color="rgba(0,0,255,0.3)"
        )
    )])
    
    fig.update_layout(
        title_text="Agent Preference Changes: Initial → Final",
        font_size=12,
        height=600
    )
    
    return fig

sankey_fig = create_sankey_chart(df)
sankey_fig.show()

## 2. Pie Chart: Agreement Frequency

In [5]:
def create_agreement_pie_chart(df):
    """
    Create pie chart showing agreement frequency across experiments
    """
    # Get unique experiments and their agreement status
    experiment_agreement = df.groupby('File')['Agreement?'].first()
    agreement_counts = experiment_agreement.value_counts()
    
    fig = px.pie(
        values=agreement_counts.values, 
        names=agreement_counts.index,
        title=f"Agreement Rate Across {len(experiment_agreement)} Experiments",
        color_discrete_map={'Yes': '#2E8B57', 'No': '#CD5C5C'}
    )
    
    fig.update_traces(
        textposition='inside',
        textinfo='percent+label',
        textfont_size=14
    )
    
    fig.update_layout(
        font_size=12,
        height=500
    )
    
    return fig, agreement_counts

agreement_fig, agreement_stats = create_agreement_pie_chart(df)
agreement_fig.show()

print(f"\nAgreement Statistics:")
for status, count in agreement_stats.items():
    percentage = count / agreement_stats.sum() * 100
    print(f"{status}: {count} experiments ({percentage:.1f}%)")


Agreement Statistics:
Yes: 8 experiments (80.0%)
No: 2 experiments (20.0%)


## 3. Bar Chart: Agreement by Principle

In [6]:
def create_agreement_by_principle_chart(df):
    """
    Create bar chart showing which principles were agreed upon when consensus was reached
    """
    # Filter only experiments with agreement
    agreed_experiments = df[df['Agreement?'] == 'Yes']
    
    if len(agreed_experiments) == 0:
        print("No experiments with agreement found.")
        return None, None
    
    # Get unique experiments and their agreed principles
    experiment_principles = agreed_experiments.groupby('File')['Agreed Principle Name'].first()
    principle_counts = experiment_principles.value_counts()
    
    # Principle name mapping for display
    principle_names = {
        'maximizing_floor': 'Floor Income',
        'maximizing_average': 'Average Income', 
        'maximizing_average_floor_constraint': 'Average + Floor Constraint',
        'maximizing_average_range_constraint': 'Average + Range Constraint'
    }
    
    # Map principle names for display
    display_names = [principle_names.get(p, p) for p in principle_counts.index]
    
    fig = px.bar(
        x=display_names,
        y=principle_counts.values,
        title=f"Principles Agreed Upon (When Consensus Reached, n={len(experiment_principles)})",
        labels={'x': 'Justice Principle', 'y': 'Number of Agreements'},
        color=principle_counts.values,
        color_continuous_scale='viridis'
    )
    
    fig.update_layout(
        xaxis_title="Justice Principle",
        yaxis_title="Number of Agreements",
        height=500,
        font_size=12,
        showlegend=False
    )
    
    # Add value labels on bars
    fig.update_traces(
        texttemplate='%{y}',
        textposition='outside'
    )
    
    return fig, principle_counts

principle_agreement_fig, principle_stats = create_agreement_by_principle_chart(df)
if principle_agreement_fig:
    principle_agreement_fig.show()
    
    print(f"\nPrinciple Agreement Statistics:")
    for principle, count in principle_stats.items():
        percentage = count / principle_stats.sum() * 100
        print(f"{principle}: {count} agreements ({percentage:.1f}%)")
else:
    print("No agreement data available for visualization.")


Principle Agreement Statistics:
maximizing_average_floor_constraint: 8 agreements (100.0%)


## 4. Bar Chart: Model Type Preference Analysis

In [12]:
def create_model_preference_analysis(df):
    """
    Create bar chart showing initial and final principle preferences by model type
    """
    # Clean data
    df_clean = df.dropna(subset=['Initial Preference Name', 'Final Preference Name'])
    
    # Principle name mapping for display
    principle_names = {
        'maximizing_floor': 'Floor Income',
        'maximizing_average': 'Average Income', 
        'maximizing_average_floor_constraint': 'Average + Floor Constraint',
        'maximizing_average_range_constraint': 'Average + Range Constraint'
    }
    
    # Map principle names
    df_clean['Initial_Display'] = df_clean['Initial Preference Name'].map(principle_names)
    df_clean['Final_Display'] = df_clean['Final Preference Name'].map(principle_names)
    
    # Create analysis for initial preferences
    initial_by_model = df_clean.groupby(['Model own', 'Initial_Display']).size().unstack(fill_value=0)
    
    # Create analysis for final preferences  
    final_by_model = df_clean.groupby(['Model own', 'Final_Display']).size().unstack(fill_value=0)
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Initial Preferences by Model Type', 'Final Preferences by Model Type'),
        vertical_spacing=0.3
        
    )
    
    colors = px.colors.qualitative.Set2
    
    # Add initial preferences bars
    for i, principle in enumerate(initial_by_model.columns):
        fig.add_trace(
            go.Bar(
                name=principle,
                x=initial_by_model.index,
                y=initial_by_model[principle],
                marker_color=colors[i % len(colors)],
                showlegend=True if i < len(initial_by_model.columns) else False
            ),
            row=1, col=1
        )
    
    # Add final preferences bars
    for i, principle in enumerate(final_by_model.columns):
        fig.add_trace(
            go.Bar(
                name=principle,
                x=final_by_model.index,
                y=final_by_model[principle],
                marker_color=colors[i % len(colors)],
                showlegend=False
            ),
            row=2, col=1
        )
    
    fig.update_layout(
        height=800,
        title_text="Principle Preferences by Model Type",
        barmode='group',
        font_size=10
    )
    
    fig.update_xaxes(title_text="Model Type", row=1, col=1)
    fig.update_xaxes(title_text="Model Type", row=2, col=1)
    fig.update_yaxes(title_text="Number of Agents", row=1, col=1)
    fig.update_yaxes(title_text="Number of Agents", row=2, col=1)
    
    return fig, initial_by_model, final_by_model

model_analysis_fig, initial_stats, final_stats = create_model_preference_analysis(df)
model_analysis_fig.show()

print("\n=== Initial Preferences by Model ===")
print(initial_stats)

print("\n=== Final Preferences by Model ===")
print(final_stats)


=== Initial Preferences by Model ===
Initial_Display                 Average + Floor Constraint  Floor Income
Model own                                                               
anthropic/claude-sonnet-4                                9             0
deepseek/deepseek-chat-v3-0324                           3             0
google/gemini-2.5-flash                                 11             0
google/gemini-2.5-pro                                    1             0
meta-llama/llama-4-maverick                              5             0
moonshotai/kimi-k2                                      11             0
openai/gpt-4.1                                           1             0
openai/gpt-4.1-mini                                      4             0
qwen/qwen3-coder                                         1             1
x-ai/grok-4                                              3             0

=== Final Preferences by Model ===
Final_Display                   Average + Floor Co

## Summary Statistics

In [8]:
def generate_summary_stats(df):
    """
    Generate comprehensive summary statistics
    """
    print("=== EXPERIMENT SUMMARY ===")
    print(f"Total experiments analyzed: {df['File'].nunique()}")
    print(f"Total agent instances: {len(df)}")
    print(f"Average agents per experiment: {len(df) / df['File'].nunique():.1f}")
    
    print("\n=== MODEL DISTRIBUTION ===")
    model_counts = df['Model own'].value_counts()
    for model, count in model_counts.items():
        percentage = count / len(df) * 100
        print(f"{model}: {count} agents ({percentage:.1f}%)")
    
    print("\n=== PREFERENCE STABILITY ===")
    df_clean = df.dropna(subset=['Initial Preference Name', 'Final Preference Name'])
    stable_prefs = (df_clean['Initial Preference Name'] == df_clean['Final Preference Name']).sum()
    print(f"Agents who kept same preference: {stable_prefs}/{len(df_clean)} ({stable_prefs/len(df_clean)*100:.1f}%)")
    print(f"Agents who changed preference: {len(df_clean) - stable_prefs}/{len(df_clean)} ({(len(df_clean) - stable_prefs)/len(df_clean)*100:.1f}%)")
    
    print("\n=== PRINCIPLE POPULARITY ===")
    print("Initial preferences:")
    initial_counts = df_clean['Initial Preference Name'].value_counts()
    for principle, count in initial_counts.items():
        percentage = count / len(df_clean) * 100
        print(f"  {principle}: {count} ({percentage:.1f}%)")
        
    print("\nFinal preferences:")
    final_counts = df_clean['Final Preference Name'].value_counts()
    for principle, count in final_counts.items():
        percentage = count / len(df_clean) * 100
        print(f"  {principle}: {count} ({percentage:.1f}%)")

generate_summary_stats(df)

=== EXPERIMENT SUMMARY ===
Total experiments analyzed: 10
Total agent instances: 50
Average agents per experiment: 5.0

=== MODEL DISTRIBUTION ===
moonshotai/kimi-k2: 11 agents (22.0%)
google/gemini-2.5-flash: 11 agents (22.0%)
anthropic/claude-sonnet-4: 9 agents (18.0%)
meta-llama/llama-4-maverick: 5 agents (10.0%)
openai/gpt-4.1-mini: 4 agents (8.0%)
deepseek/deepseek-chat-v3-0324: 3 agents (6.0%)
x-ai/grok-4: 3 agents (6.0%)
qwen/qwen3-coder: 2 agents (4.0%)
google/gemini-2.5-pro: 1 agents (2.0%)
openai/gpt-4.1: 1 agents (2.0%)

=== PREFERENCE STABILITY ===
Agents who kept same preference: 39/50 (78.0%)
Agents who changed preference: 11/50 (22.0%)

=== PRINCIPLE POPULARITY ===
Initial preferences:
  maximizing_average_floor_constraint: 49 (98.0%)
  maximizing_floor: 1 (2.0%)

Final preferences:
  maximizing_average_floor_constraint: 40 (80.0%)
  No vote: 10 (20.0%)


## Export Results

In [9]:
# Save the dataframe to CSV for further analysis
output_file = 'experiment_analysis.csv'
df.to_csv(output_file, index=False)
print(f"Data exported to {output_file}")

# Display first few rows of the final dataframe
print("\n=== Final DataFrame Preview ===")
print(df[['Agent Name', 'Model own', 'Initial Preference Name', 'Final Preference Name', 'Agreement?', 'Agreed Principle Name']].head(10))

Data exported to experiment_analysis.csv

=== Final DataFrame Preview ===
  Agent Name                       Model own  \
0    Agent_1       anthropic/claude-sonnet-4   
1    Agent_2       anthropic/claude-sonnet-4   
2    Agent_3              moonshotai/kimi-k2   
3    Agent_4       anthropic/claude-sonnet-4   
4    Agent_5     meta-llama/llama-4-maverick   
5    Agent_1             openai/gpt-4.1-mini   
6    Agent_2              moonshotai/kimi-k2   
7    Agent_3  deepseek/deepseek-chat-v3-0324   
8    Agent_4         google/gemini-2.5-flash   
9    Agent_5         google/gemini-2.5-flash   

               Initial Preference Name                Final Preference Name  \
0  maximizing_average_floor_constraint  maximizing_average_floor_constraint   
1  maximizing_average_floor_constraint  maximizing_average_floor_constraint   
2  maximizing_average_floor_constraint  maximizing_average_floor_constraint   
3  maximizing_average_floor_constraint  maximizing_average_floor_constraint   
4 